In [1]:
import requests
import re
from collections import Counter
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import json
from tqdm import tqdm
tqdm.pandas()
from itertools import chain
from user_agent import generate_user_agent

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


def create_query_words(tag, possible_values): 
    tag = re.sub("&","%26",tag)
    possible_values = [re.sub("&","%26", row) for row in possible_values]
    result = [re.sub(" ","+", tag+" "+row) for row in possible_values]
    return result

def request_links_Session(url, attempts):
    for _ in range(0, attempts):
        try:
            session = requests.Session()
            headers =  {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                        'User-Agent': str(generate_user_agent(device_type='desktop', os=('mac', 'linux'))),
                        'Connection': 'Keep-Alive',
                        'Accept-Encoding': 'gzip, deflate',
                        'Accept-Language': "en-US,en;q=0.8"}
            page = session.get(url, headers=headers, timeout = 6)
            content = bs(page.content, features="lxml")
            meta = content.find_all("div",{"class":"rg_meta"})
            if page.status_code not in [403, 404] and len(meta) > 0:
                return meta
        except Exception as err:        
            pass
        
        
def google_image_links_extractor(query_words, start_date, end_date):
    attempts = 50
    start_day, start_month, start_year = start_date
    end_day, end_month, end_year = end_date    
    query = "https://www.google.co.in/search?q="+query_words+"&tbas=0&tbs=cdr:1,cd_min:"+str(start_month)+"/"+str(start_day)+"/"+str(start_year)+",cd_max:"+str(end_month)+"/"+str(end_day)+"/"+str(end_year)+"&source=lnms&tbm=isch"
#     print(query)
    linkCollector = []
    result = request_links_Session(query, attempts)
    if result is not None:            
        for a in result:
            try:
                link = json.loads(a.text)["ou"]
                linkCollector.append(link)
            except json.JSONDecodeError as err:
                pass        
    
    return linkCollector



def query_generator(tag, start_date, end_date, post_fixes): return [row for row in tqdm(set(list(chain.from_iterable([google_image_links_extractor(query, start_date, end_date) for query in tqdm(create_query_words(tag, post_fixes), postfix=tag)]))))]


def get_links(start_date, end_date, tag_set, post_fixes):
    link_results = {}
    for tag in tag_set: link_results[tag] = query_generator(tag, start_date, end_date, post_fixes)
    return link_results

In [3]:
import pandas as pd
pathTags = "companies_new.csv"
tags = [tag[0] for tag in pd.read_csv(pathTags, header=None).values]
# tags = tags[23:]

postFixes = ['"Market share"',"Revenue", "Ebitda", "M&A",'"Users growth"','"Orders growth"',"Breakdown", "Segments"]

startDate = (20, 12, 2019) # startDay, startMonth, startYear
endDate = (21, 12, 2019) # endDay, endMonth, endYear

In [4]:
get_links(startDate, endDate, tags, postFixes)

 12%|█▎        | 1/8 [00:05<00:41,  5.86s/it, Apple]

KeyboardInterrupt: 